<a href="https://colab.research.google.com/github/almedida/thesis/blob/main/estimate_m0s_limma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
install.packages('pacman')
install.packages('BiocManager')
library(pacman)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
p_load("tidyverse", "tmvtnorm" )

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘zoo’, ‘sandwich’, ‘mvtnorm’, ‘gmm’



tmvtnorm installed



In [27]:
#load dataset on data_df
data_df = read.table("limma_pvalues.csv", header= TRUE, sep=",")

pval_raw = as.data.frame(as.matrix(data_df))
colnames(pval_raw) = c("pvals1", "pvals2")
pvals1 <- pval_raw[,1]
pvals2 <- pval_raw[,2]

head(pval_raw)


,pvals1,pvals2
,<dbl>,<dbl>
1,0.000148258,0.793365999
2,0.007329965,0.282316334
3,0.005895068,0.036705033
4,0.000007070,0.017249638
5,0.192345236,0.000236963
6,0.000072400,0.000003150


In [28]:
dim(pval_raw)

[1] 42575     2

In [29]:
###################################################################################
###################################################################################
##calc.cutoff slightly modifies the estimate.m0 function to return the appropriate
##p-value cutoff where all p-values greater than the cutoff are assumed to come
##from null cases.

calc.cutoff = function(p, B = 20, max=1){

  m <- length(p)
  m0 <- m
  bin <- c(-0.1, (1:B)/B*max)
  bin.counts=rep(0,B)

  for(i in 1:B){
    bin.counts[i]=sum((p>bin[i])&(p<=bin[i+1]))
  }

  tail.means <- rev(cumsum(rev(bin.counts))/(1:B))
  temp <- bin.counts - tail.means
  index <- min((1:B)[temp <= 0])
  cutoff2 <- (index)/B*max
  if(cutoff2 == 1) {cutoff2 <- 1-1/B}

  return(cutoff2)

}


In [30]:
cutoff_value1 = calc.cutoff(pvals1, B=20, max=1)
cutoff_value2 = calc.cutoff(pvals2, B=20, max=1)

cutoff = cbind(c(cutoff_value1), c(cutoff_value2))

colnames(cutoff) = c("cutoff_value1", "cutoff_value2")

cutoff

cutoff_value1,cutoff_value2
0.95,0.8


In [31]:
#filter dataset using cutoff points
#pvalsB and pvalsM are column names (for pvalues) of the data set for this test(to be changed to column names of the loaded data set)

# selecting pvalues in the upper-right quadrant using Histogram Based Method by Orr et al. (2012)
#defined as lambda1 = cutoffvalue1 and lambda2 = cutoffvalue2 for BS and M respectively.
#lambda1, lambda2 - probability of pvalsB and pvalsM in the upper-right quadrant respectively 


p_vals = pval_raw  %>% filter(pvals1 >=cutoff_value1, pvals2>=cutoff_value2)

#pvalsK = pval_raw[1] %>% filter(pvalue_kidney >= cutoff_value1)
#pvalsL = pval_raw[2]  %>% filter(pvalue_liver >= cutoff_value2)

# dim(p_vals)
# dim(pvalsK)
# dim(pvalsL)

# head(pvalsK)
# head(pvalsL)


In [32]:
head(p_vals)
dim(p_vals)

,pvals1,pvals2
,<dbl>,<dbl>
1,0.9700436,0.9333245
2,0.9522237,0.8221438
3,0.9500968,0.8292286
4,0.9627940,0.8703365
5,0.9675548,0.8268452
6,0.9762841,0.8324076


[1] 33  2

In [33]:
# convert selected pvalues to z values
z_val = as.data.frame(qnorm(as.matrix(p_vals), lower.tail = TRUE))
colnames(z_val) = c("zvals1", "zvals2")

head(z_val)


,zvals1,zvals2
,<dbl>,<dbl>
1,1.881435,1.5010180
2,1.666808,0.9235660
3,1.645793,0.9511214
4,1.784072,1.1279831
5,1.846012,0.9417714
6,1.982424,0.9637230


In [34]:
zvals1 <- (as.data.frame(as.matrix(z_val[,1])))
zvals2 <- (as.data.frame(as.matrix(z_val[,2])))

head(zvals1, 5)

,V1
,<dbl>
1,1.881435
2,1.666808
3,1.645793
4,1.784072
5,1.846012


In [35]:
# convert lambda(truncation points) to z values
z_val_extremums = as.data.frame(qnorm(as.matrix(cbind(c(cutoff_value1,1),c(cutoff_value2,1))), lower.tail = TRUE))

#selecting the lower bounds (minimum z values) of the bivariate z values
min_z1 <- z_val_extremums[1,1]
min_z2 <- z_val_extremums[1,2]

In [36]:

head(z_val_extremums, 2)

,V1,V2
,<dbl>,<dbl>
1,1.644854,0.8416212
2,Inf,Inf


In [37]:
###################################################################################
###################################################################################

estimate.m0s <- function(p1, p2, B=20){
  m <- length(p1)

  ##find lambda cutoffs using histogram-based method
  c1 <- calc.cutoff(p1, B=B, max=1)
  c2 <- calc.cutoff(p2, B=B, max=1)

  ##estimate m0 for experiment 1
  ind1 <- (p1>=c1)
  m0.1 <- sum(ind1)/(1-c1)

  ##estimate m0 for experiment 2  
  ind2 <- (p2>=c2)
  m0.2 <- sum(ind2)/(1-c2)


  ##estimate m00
  ind12 <- ind1 & ind2
  nA <- sum(ind12)
  #pA <- (1-c1)*(1-c2)
  #m00 <- nA/pA
  
  
  # density function for each row of the bivariate z values (x) and 
  # estimated parameters(rho)
  density = function(x, rho)
  {
    sigma = matrix(c(1, rho, rho, 1), 2, 2)
    z = dtmvnorm(x, mean = c(0,0), sigma = sigma, lower = c(min_z1, min_z2))
  }
  
  # log likelihood of the joint densities
  log_likelihood_fn = function(rho){
    
    joint_likelihood = z_val %>% split(.$zvals2) %>% map_dfr(~density(c(.$zvals1,.$zvals2),rho))    
    return(-sum(log(joint_likelihood)))
    
  }
  
  #MLE of the log likelihood function
  optimal_rho = optimize(log_likelihood_fn, lower = -1, upper = 1 )
  optimal_rho = as.data.frame(optimal_rho)
  
  #probability of a random variable greater than cutoff values 
  rho = as.numeric(optimal_rho[1])
  pA = pmvnorm(lower=c(min_z1, min_z2), upper=c(Inf, Inf), mean=c(0,0), sigma = matrix(c(1, rho, rho, 1), 2, 2))
  
  #covert zA to p-value
  #pA <- pnorm(zA, lower.tail=TRUE) 

  m00 <- nA/pA
  
  
  
  ##estimate m11
  m11 <- sum(m - m0.1 - m0.2 + m00)
  
  
  ret <- list()
  ret$ms <- c(m, m0.1, m0.2, m11, m00)
  names(ret$ms) <- c("m", "m0.1", "m0.2", "m11","m00")
  ret$cutoffs <- c(c1, c2)
  return(ret)
}

In [38]:
estimates <- estimate.m0s(pvals1, pvals2, B=20)
estimates

$ms
       m     m0.1     m0.2      m11      m00 
42575.00  7440.00 17060.00 28116.63 10041.63 

$cutoffs
[1] 0.95 0.80

In [ ]:
estimates <- estimate.m0s(pvals1, pvals2, B=20)
estimates
#write.csv(estimate.m0s, estimate_m0s.csv)

In [ ]:
mos <- as.matrix(estimates)
write.csv(mos, "estimates.csv")

In [ ]:
###########
#results for t-test
m = 42575
m0(1) = 8000
m0(2) = 17120
m11 = 20821
m00 = 3366

cutoffs: 0.8, 0.9

############
#results for limma pvalues

m.01 = 7439.9
m.02 = 17060
m11 = 28116.6
m00 = 10041.6
cutoffs: 0.95, 0.8

